### Multivariate time series prediction using MLP with hyperparameter optimization

In this notebook, we use **Optuna** to find the optimum values of hyperparameters. Optuna is a package for optimizing hyperparameters. In this notebook we specifically optimize the values of **learning rate** and **weight decay**.

Optuna is a python package specifially designed for hyperparameter tuning. We need to define a range of possible values for each of the hyperparameters. And optuna will try different parameter values with the model to minimize the validation loss after for specified number of experiments.


In [16]:
import torch
import numpy as np

import pandas as pd
from sklearn.preprocessing import MinMaxScaler
!pip install optuna
import optuna

In [17]:
from google.colab import drive
drive.mount("/content/drive", force_remount = True)

Mounted at /content/drive


In [18]:
import os
os.chdir('/content/drive/MyDrive/wsh_2025/Project Codes/input datafiles/')

In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'device: {device}')

# set seed
torch.manual_seed(42)
np.random.seed(42)

if torch.cuda.is_available():
  torch.cuda.manual_seed(42)
  torch.cuda.manual_seed_all(42)

device: cpu


In [20]:
input_file = '11447650_filled_daily_data.csv'

In [21]:
class RiverData(torch.utils.data.Dataset):

  # first function is the __init__() function, which loads data from input file
  def __init__(self, df, target, datecol, seq_len, pred_len):
    self.df = df
    self.datecol = datecol
    self.target = target
    self.seq_len = seq_len # length of single sequence of input data
    self.pred_len = pred_len # length of forward-forecasting sequence

  # second function sets the index to the date column
  def setIndex(self):
    self.df.set_index(self.datecol, inplace = True)

  # third function defines the length of the input dataset, minus the
  # training sequence length minus the prediction length
  def __len__(self):
    return len(self.df) - self.seq_len - self.pred_len

  # fourth function returns a typle of a feature and a label, which is used for
  # model training. for time series, the feature is the past values for
  # training and the label are the future values to be predicted
  def __getitem__(self, idx):

    # raises warning if not enough data at index idx
    if len(self.df) <= (idx + self.seq_len + self.pred_len):
      raise IndexError(
          f'Index {idx} out of bounds for dataset size {len(self.df)}')

    # pulls values for training sequence and assigns to df_piece
    df_piece = self.df[idx:idx + self.seq_len].values

    # converts df_piece into a tensor data type
    feature = torch.tensor(df_piece, dtype = torch.float32)

    # pulls the target data
    label_piece = self.df[self.target][
        idx + self.seq_len:
        idx + self.seq_len + self.pred_len].values

    # converts label_piece to tensor data type
    label = torch.tensor(label_piece, dtype = torch.float32)

    return (feature, label)

Normalizing data:

In [22]:
df = pd.read_csv(input_file)
df.drop('SSD_st_d', axis = 1, inplace = True) # drop ssd
df.drop('Precip_in', axis = 1, inplace = True) # drop wrong precip

raw_df = df.drop('DATE', axis = 1, inplace = False)
scaler = MinMaxScaler()

# apply transformations and make new df
df_scaled = scaler.fit_transform(raw_df)

df_scaled = pd.DataFrame(df_scaled, columns = raw_df.columns)
df_scaled['DATE'] = df['DATE']

df = df_scaled

In [23]:
train_size = int(0.7 * len(df))
test_size = int(0.2 * len(df))
val_size = len(df) - train_size - test_size

seq_len = 13
pred_len = 1
num_features = 3

# remember: single asterix unpacks list
common_args = ['SSC_mg_L', 'DATE', seq_len, pred_len]

# pulled from beginning of df to train_size
train_dataset = RiverData(df[:train_size], *common_args)
train_dataset.setIndex()

# pulled from the end of train_size to the end of val_size
val_dataset = RiverData(df[train_size: train_size + val_size], *common_args)
val_dataset.setIndex()

# pulled from the end of train + val to the end of the df
test_dataset = RiverData(df[train_size + val_size: len(df)], *common_args)
test_dataset.setIndex()

In [24]:
# number of training examples used in one iteration to update model params
BATCH_SIZE = 492

SHUFFLE = False # order matters for time series
DATA_LOAD_WORKERS = 1

# learning rate determines step size at which params are updated while training
#learning_rate = 0.001

# weight decay adds penalty to loss function based on magnitude of model
# weights, preventing overreliance on single parameter
#weight_decay = 0.0001

In [25]:
from torch.utils.data import DataLoader

# remember: double asterix unpacks dictionary
common_args = {'batch_size': BATCH_SIZE, 'shuffle': SHUFFLE}
train_loader = DataLoader(train_dataset, **common_args)
val_loader = DataLoader(val_dataset, **common_args)
test_loader = DataLoader(test_dataset, **common_args)

Defining PyTorch LSTM model:

In [26]:
class BasicLSTMNetwork(torch.nn.Module):
  # __init__() function sets up layers and defines model params
  def __init__(self, seq_len, pred_len):
    # call base class constructor
    super().__init__()
    self.seq_len = seq_len
    self.pred_len = pred_len
    self.num_features = num_features
    self.n_layers = 1

    # define size of hidden state
    self.n_hidden = 128

    # define layers for combining across time series
    self.lstm1 = torch.nn.LSTM(input_size = self.num_features,
                               hidden_size = self.n_hidden,
                               num_layers = self.n_layers,
                               batch_first = True)
    self.relu = torch.nn.ReLU()
    self.fc1 = torch.nn.Linear(self.n_hidden * self.seq_len, self.pred_len)

  def init_hidden(self, batchsize):
    device = next(self.parameters()).device
    hidden_state = torch.zeros(self.n_layers,
                               batchsize,
                               self.n_hidden,
                               device = device)
    cell_state = torch.zeros(self.n_layers,
                             batchsize,
                             self.n_hidden,
                             device = device)
    return hidden_state, cell_state

  # forward() function defines how forward pass computation operates
  # gradients are stored inside FC layer objects
  # each training example needs the old gradient erased
  def forward(self, x):
    batchsize, seqlen, featlen = x.size()
    self.hidden_states = self.init_hidden(batchsize)
    lstm_out, self.hidden_states = self.lstm1(x, self.hidden_states)
    lstm_out = lstm_out.contiguous().view(batchsize, -1)
    lstm_out = self.relu(lstm_out)
    lstm_out = self.fc1(lstm_out)
    return lstm_out

In [27]:
loss = torch.nn.MSELoss()

In [28]:
# model param check 2
for i, (f, l) in enumerate(train_loader):
  print('features shape: ', f.shape)
  print('labels shape: ', l.shape)
  break

features shape:  torch.Size([492, 13, 3])
labels shape:  torch.Size([492, 1])


In [29]:
# define metrics
import numpy as np
import matplotlib.pyplot as plt
epsilon = np.finfo(float).eps

def wape_function(y, y_pred):
    # Weighted Average Percentage Error metric in the interval [0; 100]
    y = np.array(y)
    y_pred = np.array(y_pred)
    nominator = np.sum(np.abs(np.subtract(y, y_pred)))
    denominator = np.add(np.sum(np.abs(y)), epsilon)
    wape = np.divide(nominator, denominator) * 100.0
    return wape

def nse_function(y, y_pred):
    y = np.array(y)
    y_pred = np.array(y_pred)
    return (1-(np.sum((y_pred-y)**2)/np.sum((y-np.mean(y))**2)))


def evaluate_model(model, data_loader, plot=False):
    # following line disables dropout and batch normalization if they
    # are part of the model.

    model.eval()
    all_inputs = torch.empty((0, seq_len, num_features))
    all_labels = torch.empty(0, pred_len)
    for inputs, labels in data_loader:
        all_inputs = torch.vstack((all_inputs, inputs))
        all_labels = torch.vstack((all_labels, labels))

    with torch.no_grad():
        all_inputs = all_inputs.to(device)
        outputs = model(all_inputs).detach().cpu()
        avg_val_loss = loss(outputs, all_labels)
        nse = nse_function(all_labels.numpy(), outputs.numpy())
        wape = wape_function(all_labels.numpy(), outputs.numpy())

    print(f'NSE : {nse}', end=' ')
    print(f'WAPE : {wape}', end=' ')
    print(f'Validation Loss: {avg_val_loss}')
    model.train()

    if plot is True:
        plt.figure(figsize=(16, 6))
        plt.plot(np.array(all_labels.cpu()[:400]), color='green', label='observations')
        plt.plot(np.array(outputs.cpu()[:400]), color='red', linestyle='-', label='predictions')
        metrics_text = f"NSE: {nse:.3f}\nWAPE: {wape:.3f}"
        plt.text(0.28, 0.97, metrics_text, transform=plt.gca().transAxes,
        fontsize=12, verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
        plt.legend()
        plt.show()

    return avg_val_loss



In [30]:
def objective(trial):
    # define the search space of the hyper-parameters -- optuna uses
    # bayesian optimization to find the optimal values of the hyperparameters.
    learning_rate = trial.suggest_loguniform('lr', 1e-4, 1e-2)
    weight_decay = trial.suggest_loguniform('weight_decay', 1e-5, 1e-2)

    model = BasicLSTMNetwork(seq_len, pred_len)
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(),
                                 lr = learning_rate,
                                 weight_decay=weight_decay)

    # shoot for 50 next
    num_epochs = 300

    best_val_loss = float('inf')

    # keep this between 5-10
    patience = 10

    for epoch in range(num_epochs):
        model.train()
        epoch_loss = []
        for batch_idx, (inputs, labels) in enumerate(train_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            loss_val = loss(outputs, labels)

            # calculate gradients for back propagation
            loss_val.backward()

            # update the weights based on the gradients
            optimizer.step()

            # reset the gradients, avoid gradient accumulation
            optimizer.zero_grad()
            epoch_loss.append(loss_val.item())

        avg_train_loss = sum(epoch_loss)/len(epoch_loss)
        print(f'Epoch {epoch+1}: Training Loss: {avg_train_loss}', end=' ')
        avg_val_loss = evaluate_model(model, val_loader)

        # check for improvement
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            epochs_no_improve = 0
            # save the best model
            torch.save(model.state_dict(), 'best_model_trial.pth')
        else:
            epochs_no_improve += 1
            if epochs_no_improve == patience:
                print('Early stopping!')
                # load the best model before stopping
                model.load_state_dict(torch.load('best_model_trial.pth'))
                break

        # report intermediate objective value
        trial.report(best_val_loss, epoch)

        # handle pruning based on the intermediate value
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return best_val_loss

study = optuna.create_study(direction='minimize')

study.optimize(objective, n_trials = 200)

print('Number of finished trials:', len(study.trials))
print('Best trial:')
trial = study.best_trial

print('  Value (Best Validation Loss):', trial.value)
print('  Params:')
for key, value in trial.params.items():
    print(f'    {key}: {value}')




[I 2025-08-07 19:07:08,372] A new study created in memory with name: no-name-db8c71d1-eee5-4c75-828c-5cbe829e8850
/tmp/ipython-input-420348275.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('lr', 1e-4, 1e-2)
/tmp/ipython-input-420348275.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform('weight_decay', 1e-5, 1e-2)


Epoch 1: Training Loss: 0.006363313823385397 NSE : -5.6298651695251465 WAPE : 155.39746930437138 Validation Loss: 0.001434034900739789
Epoch 2: Training Loss: 0.005044633580837399 NSE : -11.320901870727539 WAPE : 215.37152029351066 Validation Loss: 0.002665001666173339
Epoch 3: Training Loss: 0.004715505107014906 NSE : -4.200301170349121 WAPE : 137.51651622855474 Validation Loss: 0.0011248211376369
Epoch 4: Training Loss: 0.004477015201700851 NSE : -2.203913688659668 WAPE : 106.47596256071377 Validation Loss: 0.0006930041126906872
Epoch 5: Training Loss: 0.004328573337261332 NSE : -3.4340620040893555 WAPE : 127.41472139417564 Validation Loss: 0.0009590843692421913
Epoch 6: Training Loss: 0.0041492801829008386 NSE : -3.1787524223327637 WAPE : 123.84272998269788 Validation Loss: 0.0009038609568960965
Epoch 7: Training Loss: 0.00399313164234627 NSE : -1.6568248271942139 WAPE : 97.31419399220363 Validation Loss: 0.000574669218622148
Epoch 8: Training Loss: 0.003911273626727052 NSE : -1.634

[I 2025-08-07 19:15:47,848] Trial 0 finished with value: 3.8298974686767906e-05 and parameters: {'lr': 0.00025955235006424986, 'weight_decay': 0.0001198159412577847}. Best is trial 0 with value: 3.8298974686767906e-05.


Epoch 300: Training Loss: 0.0009943796385414316 NSE : 0.8229352235794067 WAPE : 21.156523733088743 Validation Loss: 3.8298974686767906e-05
Epoch 1: Training Loss: 0.008697737357579172 NSE : -12.021366119384766 WAPE : 222.62673698692961 Validation Loss: 0.002816511783748865
Epoch 2: Training Loss: 0.0039337555408565095 NSE : -0.5768553018569946 WAPE : 62.13952596360301 Validation Loss: 0.0003410726203583181
Epoch 3: Training Loss: 0.004087241126399022 NSE : -2.309868574142456 WAPE : 103.57204144170436 Validation Loss: 0.000715922040399164
Epoch 4: Training Loss: 0.0043949831306235865 NSE : -0.35761773586273193 WAPE : 66.90138625419524 Validation Loss: 0.00029365174123086035
Epoch 5: Training Loss: 0.0035991910699522123 NSE : -1.2149837017059326 WAPE : 89.7260261407934 Validation Loss: 0.00047909931163303554
Epoch 6: Training Loss: 0.0030263619555626065 NSE : -0.4654346704483032 WAPE : 72.1773321381668 Validation Loss: 0.00031697243684902787
Epoch 7: Training Loss: 0.0026098334583366523 

[I 2025-08-07 19:16:57,583] Trial 1 finished with value: 2.6064595658681355e-05 and parameters: {'lr': 0.005331202123074718, 'weight_decay': 0.00015174173660036834}. Best is trial 1 with value: 2.6064595658681355e-05.


...

Epoch 2: Training Loss: 0.003982955922765541 NSE : -1.7529146671295166 WAPE : 91.8699881178212 Validation Loss: 0.0005954533698968589
Epoch 1: Training Loss: 0.010060397267807275 NSE : 0.2465943694114685 WAPE : 33.419272060203035 Validation Loss: 0.0001629610633244738
Epoch 2: Training Loss: 0.0046199790958780795 NSE : -0.08703482151031494 WAPE : 56.679894102687044 Validation Loss: 0.0002351248258491978
Epoch 3: Training Loss: 0.003704337985254824 NSE : 0.32004815340042114 WAPE : 40.786917095745345 Validation Loss: 0.00014707306399941444
Epoch 4: Training Loss: 0.0033654498256510124 NSE : 0.004305660724639893 WAPE : 56.914181484646974 Validation Loss: 0.00021536795247811824


[I 2025-08-07 19:58:28,347] Trial 198 pruned. 


Epoch 5: Training Loss: 0.0028873950359411538 NSE : 0.33600884675979614 WAPE : 45.06213337224573 Validation Loss: 0.0001436207676306367


[I 2025-08-07 19:58:29,803] Trial 199 pruned. 


Epoch 1: Training Loss: 0.0053932967957734945 NSE : -1.4594135284423828 WAPE : 93.51231368952075 Validation Loss: 0.0005319693009369075
Number of finished trials: 200
Best trial:
  Value (Best Validation Loss): 1.6306281395372935e-05
  Params:
    lr: 0.007240183391544309
    weight_decay: 1.3647269057168989e-05


In [31]:
import optuna.visualization as vis

# optimization history
fig1 = vis.plot_optimization_history(study)
fig1.write_html("optimization_history_lstm.html")